# example1

The simulation of financial markets through the use of agent-based models is an increasingly popular technique to understand the microstructure of their dynamics from the bottom up. In this notebook, we'll look at an example market simulation.

In [1]:
# Usual imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

Looking at this paper: https://www.nature.com/articles/srep08399.pdf ...

In [62]:
class agentens:
    def __init__(self, setup : dict):
        """A class for an ensemble of agents which
        can be evolved in time."""
        self.setup = setup
        self._invest_horizons = None
        self.stocks_held = np.random.randint(
            0,
            self.setup["Nstocks"] + 1,
            size=(
                self.setup["Nagents"],
                self.setup["Nreals"],
            ),
        )
        self.ln_returns = np.zeros(
            (self.setup["Nagents"], self.setup["Nreals"])
        )
    @property
    def invest_horizons(self):
        if self._invest_horizons is None:
            self.invest_horizon_times = np.random.exponential(
                np.tensordot(
                    1.0 
                    / np.random.gamma(
                        self.setup["GammaShape"],
                        self.setup["GammaScale"],
                        size=self.setup["Nagents"],
                    ),
                    np.ones(self.setup["Nreals"]),
                    axes=0,
                ),
                size=(
                    self.setup["Nagents"],
                    self.setup["Nreals"],
                )
            )
            self._invest_horizons = (
                self.invest_horizon_times 
                / self.setup["tstep"]
            ).astype(int)
            self.max_horizon = self._invest_horizons.max()
            self.max_horizon_time = self.invest_horizon_times.max()
            self.price_diff_history = np.zeros(
                (self.max_horizon, self.setup["Nstocks"], self.setup["Nreals"])
            )
        return self._invest_horizons
    def iterate(self, prices : np.ndarray):
        """Take the ensemble a step forward in time by
        asking each agent to make a buy-sell-hold decision."""
        draws = np.random.uniform(size=(self.setup["Nagents"], self.setup["Nreals"]))
        self.decisions = (
            0
            + 1 * (self.buyprobs > draws)
            - 1 * (self.sellprobs + self.buyprobs > draws >= self.buyprobs)
        )
        self.ln_returns = np.cumsum(np.log(self.price_diff_history), axis=0)

In [63]:
class marketens:
    def __init__(self, setup : dict):
        """A class for an ensemble of simulated market 
        prices which can be evolved in time."""
        self.setup = setup
        self._ae = None
    @property
    def ae(self):
        if self._ae is None:
            self._ae = agentens(self.setup)
            self.prices = np.apply_along_axis(
                lambda s: 
                np.histogram(s, bins=np.arange(0, self.setup["Nstocks"] + 1, 1))[0],
                0, 
                self._ae.stocks_held,
            )
        return self._ae
    def iterate(self):
        """Take the market prices (and the ensemble) 
        a step forward in time."""
        self.ae.iterate(self.prices)
        self.prices = self.ae.decisions

In [64]:
k = 100.0
setup = {
    "Nstocks" : 100,
    "Nagents" : 1000,
    "Nreals" : 10,
    "GammaShape" : k,
    "GammaScale" : 1.0 / k,
    "tstep" : 0.1,
}
me = marketens(setup)